# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [1]:
from resolve import *

Setting working directory to: /home/robsyc/Desktop/thesis/MB-VAE-DTI


In [46]:
from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/summer/version3/baseline_kiba_rand")

if "metadata.data.provenance_cols" in df.columns:
    df["metadata.data.provenance_cols"] = df["metadata.data.provenance_cols"].apply(lambda x: x[0])
if "metadata.data.drug_features" in df.columns:
    df = df.drop(columns=["metadata.data.drug_features", "metadata.data.target_features"])

df

2025-07-27 15:51:32,458 - mb_vae_dti.validating.analysis - INFO - Successfully loaded 307 results with 37 columns


,experiment_name,best_val_loss,best_epoch,trainable_params,timestamp,metadata.training.max_epochs,metadata.training.early_stopping_patience,metadata.training.scheduler,metadata.training.learning_rate,metadata.data.load_in_memory,...,metrics.val/ci,metrics.val/mse,metrics.val/pearson,metrics.val/r2,metrics.val/rmse,timing.total_training_time,timing.avg_time_per_epoch,timing.total_epochs,timing.min_epoch_time,timing.max_epoch_time
0,baseline_KIBA_rand_b00c0057_1753372452,0.187791,64,1491072,1.753375e+09,100,10,one_cycle,0.0020,True,...,0.844453,0.188922,0.851549,0.717682,0.546102,2341.595423,31.221272,75,30.179613,32.086768
1,baseline_KIBA_rand_b02c0036_1753324117,0.170744,80,12640256,1.753325e+09,100,10,one_cycle,0.0020,True,...,0.848709,0.176710,0.862845,0.736008,0.531286,1309.302848,14.387943,91,13.295473,14.580125
2,baseline_KIBA_rand_b00c0054_1753366167,0.173709,53,6328320,1.753370e+09,100,10,one_cycle,0.0020,True,...,0.849146,0.178502,0.861195,0.733325,0.536485,3509.865906,54.841655,64,53.838365,54.389980
3,baseline_KIBA_rand_b00c0034_1753305992,0.193858,57,6333952,1.753313e+09,100,10,one_cycle,0.0020,True,...,0.843936,0.199380,0.845674,0.702135,0.556256,6648.295969,97.769058,68,96.152058,97.667458
4,baseline_KIBA_rand_b01c0044_1753361072,0.173366,74,5012992,1.753362e+09,100,10,one_cycle,0.0005,True,...,0.849650,0.179769,0.861777,0.731438,0.533329,1244.764844,14.644292,85,13.604424,14.557539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,baseline_KIBA_rand_b01c0005_1753226937,0.183169,77,4226816,1.753229e+09,100,10,one_cycle,0.0020,True,...,0.848749,0.183559,0.855961,0.725778,0.546125,1568.228224,17.820775,88,16.622552,18.346150
303,baseline_KIBA_rand_b04c0005_1753557498,0.183941,81,4224000,1.753559e+09,100,10,one_cycle,0.0005,True,...,0.843372,0.185452,0.853608,0.722949,0.587740,1215.817166,13.215404,92,11.924726,13.773092
304,baseline_KIBA_rand_b01c0008_1753237530,0.253937,82,17638144,1.753240e+09,100,10,one_cycle,0.0005,True,...,0.822192,0.258884,0.799952,0.613247,0.661315,2092.216421,22.496951,93,21.488367,22.466134
305,baseline_KIBA_rand_b00c0058_1753374826,0.165948,77,20780544,1.753380e+09,100,10,one_cycle,0.0020,True,...,0.853027,0.170201,0.867935,0.745727,0.502491,4790.988984,54.443057,88,53.318611,54.444476


In [47]:
varying_attrs = get_varying_attributes(df)
varying_attrs.remove('metadata.logging.experiment_name')
varying_attrs.extend(["best_val_loss", 'best_epoch', 'trainable_params', "timing.total_training_time"])
varying_attrs

2025-07-27 15:51:33,206 - mb_vae_dti.validating.analysis - INFO - Found 7 varying attributes: ['metadata.training.learning_rate', 'metadata.data.batch_size', 'metadata.logging.experiment_name', 'metadata.model.encoder_kwargs.hidden_dim', 'metadata.model.encoder_kwargs.n_layers', 'metadata.model.embedding_dim', 'metadata.model.encoder_type']


['metadata.training.learning_rate',
 'metadata.data.batch_size',
 'metadata.model.encoder_kwargs.hidden_dim',
 'metadata.model.encoder_kwargs.n_layers',
 'metadata.model.embedding_dim',
 'metadata.model.encoder_type',
 'best_val_loss',
 'best_epoch',
 'trainable_params',
 'timing.total_training_time']

In [45]:
subdf = get_top_performers(df, n_top=150)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(df["timing.total_training_time"].mean())
subdf

4578.8781306252895


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,aggregator_type,best_val_loss,best_epoch,trainable_params,total_training_time
290,0.0005,16,256,3,1024,resnet,concat,0.154958,85,10555136,7962.247344
422,0.0010,32,512,1,1024,resnet,concat,0.155305,83,16484864,2606.612936
300,0.0010,16,256,3,1024,resnet,concat,0.156679,93,10555136,8547.191034
379,0.0020,16,256,1,1024,resnet,concat,0.157262,95,5294336,4868.378374
229,0.0005,32,256,2,768,resnet,concat,0.157600,90,7596544,3258.366287
...,...,...,...,...,...,...,...,...,...,...,...
372,0.0010,16,256,1,512,resnet,concat,0.169706,99,4637952,4928.664393
371,0.0010,32,256,3,512,resnet,concat,0.169796,81,9898752,3664.909983
317,0.0020,32,128,3,768,resnet,attentive,0.169833,96,2770818,4211.608700
269,0.0010,64,256,2,512,resnet,attentive,0.169851,95,6874114,1942.267176


In [48]:
subdf = get_top_performers(df, n_top=150)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(df["timing.total_training_time"].mean())
subdf

2944.244171038513


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,best_val_loss,best_epoch,trainable_params,total_training_time
84,0.0005,16,512,4,512,resnet,0.162432,87,20517888,6429.907681
182,0.0010,32,512,3,512,resnet,0.162529,65,16316416,2047.399992
280,0.0005,64,512,4,512,resnet,0.163233,71,20517888,1269.630827
157,0.0020,64,512,4,512,resnet,0.163496,71,20517888,1283.131603
137,0.0010,32,512,4,512,resnet,0.163650,69,20517888,2895.877781
...,...,...,...,...,...,...,...,...,...,...
67,0.0010,32,256,2,512,transformer,0.186785,77,4095232,2705.758324
296,0.0020,64,512,3,768,transformer,0.186998,91,17375488,1963.834045
92,0.0005,32,128,4,512,resnet,0.187073,56,1984512,2047.335534
277,0.0010,32,128,2,1024,resnet,0.187131,69,1588736,2199.116183
